In [ ]:
# Emergency Detection System - Training & Evaluation Notebook

import sys
import os
import logging
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- PATHING SETUP ---
# Ensures the notebook can find the 'shallnotcrash' package.
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    logging.info(f"Added project root to Python path: {project_root}")

# --- IMPORTS ---
from shallnotcrash.emergency.synthetic_data import generate_training_data
from shallnotcrash.emergency.core import EMERGENCY_COORDINATOR
from shallnotcrash.emergency.analyzers.pattern_recognizer import EmergencyPattern, PATTERN_RECOGNIZER

# --- CONFIGURATION ---
NUM_TRAINING_SAMPLES = 10000
NUM_TEST_SAMPLES = 2000
MODEL_DIR = os.path.join(project_root, 'models')
MODEL_FILENAME = "c172p_pattern_recognizer_v1.joblib"
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_FILENAME)

def train_model():
    """Train the Emergency Pattern Recognizer."""
    logging.info("--- Starting Emergency Pattern Recognizer Training Protocol ---")
    
    # --- Step 1: Ensure model directory exists ---
    if not os.path.exists(MODEL_DIR):
        logging.info(f"Model directory not found. Creating directory at: {MODEL_DIR}")
        os.makedirs(MODEL_DIR)
    
    # --- Step 2: Generate Training Data ---
    logging.info(f"Generating {NUM_TRAINING_SAMPLES} samples for training...")
    training_data = generate_training_data(NUM_TRAINING_SAMPLES)
    logging.info("Training data generation complete.")
    
    # --- Step 3: Train the Models ---
    logging.info("Passing data to the Pattern Recognizer for training...")
    # The train_models method is part of the PatternRecognizer class
    PATTERN_RECOGNIZER.train_models(training_data)
    logging.info("Model training complete.")
    
    # --- Step 4: Save the Trained Models ---
    logging.info(f"Saving trained model artifact to: {MODEL_PATH}")
    PATTERN_RECOGNIZER.save_models(MODEL_PATH)
    logging.info("--- Training Protocol Finished Successfully ---")

def evaluate_system():
    """Evaluate the integrated emergency detection system."""
    logging.info("--- Starting Integrated System Evaluation ---")
    
    # --- Step 1: Validate Coordinator Status ---
    if not EMERGENCY_COORDINATOR.is_loaded:
        logging.error("FATAL: The EMERGENCY_COORDINATOR is not operational. Evaluation cannot proceed.")
        return
    
    # --- Step 2: Generate Unseen Test Data ---
    logging.info(f"Generating {NUM_TEST_SAMPLES} new, unseen test samples...")
    test_data = generate_training_data(NUM_TEST_SAMPLES)
    true_labels = []
    predicted_labels = []
    
    # --- Step 3: Execute Detection Pipeline ---
    logging.info("Running test samples through the integrated EMERGENCY_COORDINATOR...")
    for i, sample in enumerate(test_data):
        if (i + 1) % 200 == 0:
            logging.info(f" Processing sample {i+1}/{NUM_TEST_SAMPLES}...")
        
        # The true label is the integer value of the emergency pattern
        true_labels.append(sample['pattern_label'])
        
        # The coordinator returns a full PatternResult object
        result = EMERGENCY_COORDINATOR.detect(telemetry=sample['telemetry'])
        
        # The predicted label is the integer value from the result's pattern_type
        predicted_labels.append(result.pattern_type.value)
    
    # --- Step 4: Analyze and Display Results ---
    logging.info("Evaluation complete. Generating performance reports...")
    
    # Get the integer values for scikit-learn functions
    report_labels_int = sorted([p.value for p in EmergencyPattern])
    # Get the string names for clear, human-readable plot labels
    plot_labels_str = [EmergencyPattern(i).name for i in report_labels_int]
    
    # Classification Report
    print("\n" + "="*50)
    print(" CLASSIFICATION REPORT (INTEGRATED SYSTEM)")
    print("="*50)
    report = classification_report(
        true_labels,
        predicted_labels,
        labels=report_labels_int,
        target_names=plot_labels_str,  # Use string names for the report
        zero_division=0
    )
    print(report)
    print("="*50 + "\n")
    
    # Generate confusion matrix with integer labels
    cm = confusion_matrix(true_labels, predicted_labels, labels=report_labels_int)
    
    # Display confusion matrix with clear string labels
    plt.figure(figsize=(14, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=plot_labels_str, yticklabels=plot_labels_str)
    plt.title('Confusion Matrix for Integrated Emergency Detection', fontsize=16)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.xticks(rotation=45, ha="right")  # Improved rotation for readability
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    # Uncomment the function you want to run:
    
    # Train the model
    # train_model()
    
    # Evaluate the system (assumes model is already trained)
    evaluate_system()

print("Notebook setup complete. Use train_model() to train or evaluate_system() to evaluate.")